# Usable Data Mask (UDM2) Cloud Detection within an AOI

This guide is a follow up to the [UDM2 Cloud Detection](udm2_clouds.ipynb) notebook. Please refer to that notebook for further details on specifications and usage of the UDM2 asset. In this notebook, we apply cloud detection to a specific area of interest (AOI).

The `ortho_udm2` asset is available for `PSScene` 4-band items created after 2018-08-01. Therefore, our search should be limited to these items and this date range. In this notebook, we focus on `PSScene` imagery taken within the month of April, 2019.

## Import dependencies

In [ ]:
import datetime
import json
import os
import time

from datetime import datetime
from planet import Auth
from planet import Session, DataClient, OrdersClient, data_filter
import rasterio
from rasterio.plot import show
import requests

In [ ]:
# if your Planet API Key is not set as an environment variable, you can paste it below
API_KEY = os.getenv('PL_API_KEY', 'PASTE_YOUR_KEY_HERE')

client = Auth.from_key(API_KEY)

## Finding clear imagery

One of the benefits of accurate and automated cloud detection is that it allows users to filter out images that don't meet a certain quality threshold. Planet's Data API allows users to [search](https://developers.planet.com/docs/apis/data/searches-filtering/) based on the value of the imagery metadata.
    
Planet's cloud detection algorithm classifies every pixel into one of six different categories, each of which has a corresponding metadata field that reflects the percentage of data that falls into the category.

| Class | Metadata field |
| --- | --- |
| clear | `clear_percent` |
| snow | `snow_ice_percent` |
| shadow | `shadow_percent` |
| light haze | `light_haze_percent` |
| heavy haze| `heavy_haze_percent` |
| cloud | `cloud_percent` |

The [UDM2 Cloud Detection](udm2_clouds.ipynb) notebook provides examples for how to perform searches using the Planet Python Client command-line interface.

The following example will show how use Planet's Python Client to perform a search for PSScene images that are 70-95% clear (not totally clear so the UDM2 has some interesting content), taken within the month of April, 2019, and within an AOI.

### UDM2.1

After November 2023, with the launch of UDM2.1, the heavy haze class has been deprecated. A new computer vision model trained at Planet returns more accurate class masks, and includes only a single haze class. For backwards compatibility, `light_haze` will store this haze detection, whereas `heavy_haze` will return zero for all pixels going forward.

In [ ]:
# define the aoi for imagery
aoi = {"geometry": 
       {
        "type":"Polygon",
        "coordinates":
            [[
                [25.42429478260258, 1.0255377823058893],
                [25.592960813580472, 1.0255377823058893],
                [25.592960813580472, 1.1196578801254304],
                [25.42429478260258, 1.1196578801254304],
                [25.42429478260258, 1.0255377823058893]
            ]]
        }
      }


item_types = ['PSScene']

In [ ]:
# filter by our gemoetry defined above
geom_filter = data_filter.geometry_filter(aoi)

# clear_percent between 70 and 95, inclusive
clear_percent_filter = data_filter.range_filter('clear_percent', None, None, 70, 95)

# imagery acquired between April 1st and May 1st 2019
date_range_filter = data_filter.date_range_filter("acquired", datetime(month=4, day=1, year=2019), datetime(month=5, day=1, year=2019))

# combine all these filters together
combined_filter = data_filter.and_filter([geom_filter, clear_percent_filter, date_range_filter])

In [ ]:
async with Session() as sess:
    cl = DataClient(sess)
    request = await cl.create_search(name='udm2_clouds_aoi',search_filter=combined_filter, item_types=item_types)

In [ ]:
request

In [ ]:
# Search the Data API
async with Session() as sess:
    cl = DataClient(sess)
    items = cl.run_search(search_id=request['id'])
    item_list = [i async for i in items]

In [ ]:
print(len(item_list))

In [ ]:
# Let's look at the first item in our results list
item = item_list[0]
item_id = item_list[0]['id']
item_id

## The `udm2` asset

In addition to metadata for filtering, the `udm2` asset provides a pixel-by-pixel map that identifies the classification of each pixel. See the [UDM2 Cloud Detection](udm2_clouds.ipynb) notebook for an example map.

The `udm2` structure is to use a separate band for each classification type. Band 2, for example, indicates that a pixel is snowy when its value is 1, band 3 indicates shadow and so on. "

The following Python will download the data above and then display pixels that fall into a certain classifications.

First, we need to activate the asset that we've identified:

In [ ]:
async def download_asset(item_type, item_id, asset_type, destination_folder, overwrite=True):
    cl = DataClient(Session())

    # Get Asset
    asset_desc = await cl.get_asset(item_type_id=item_type, item_id=item_id, asset_type_id=asset_type)

    # Activate Asset
    await cl.activate_asset(asset=asset_desc)

    # Wait for asset to become active
    print('Awaiting asset activation...', end=' ')
    asset = await cl.wait_asset(asset_desc)

    # Download Asset
    print('Done. Downloading asset.')
    asset_path = await cl.download_asset(asset, directory=destination_folder, overwrite=overwrite)
    
    return asset_path

In [ ]:
data_folder = 'data'

The following cells will activate and download the desired assets. However, activation can take several minutes to finish.

In [ ]:
# Activate and download UDM asset
udm2_file = await download_asset('PSScene', item_id, 'ortho_udm2', data_folder)

In [ ]:
# Activate and download image asset
img_file = await download_asset('PSScene', item_id, 'ortho_analytic_4b', data_folder)

### Visualize Image and UDM2

In [ ]:
with rasterio.open(udm2_file) as src:
    shadow_mask = src.read(3).astype(bool)
    cloud_mask = src.read(6).astype(bool)
    
show(shadow_mask, title="shadow", cmap="binary")
show(cloud_mask, title="cloud", cmap="binary")

In [ ]:
mask = shadow_mask + cloud_mask
show(mask, title="mask", cmap="binary")

In [ ]:
with rasterio.open(img_file) as src:
    profile = src.profile
    img_data = src.read([3, 2, 1], masked=True) / 10000.0 # apply RGB ordering and scale down

In [ ]:
show(img_data, title=item_id)

In [ ]:
img_data.mask = mask
img_data = img_data.filled(fill_value=0)

In [ ]:
show(img_data, title="masked image")

The image stored in `img_data` now has cloudy / cloud-shadowy pixels masked out and can be saved or used for analysis.